● 관련 라이브러리 불러오기

In [7]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from sklearn.model_selection import train_test_split

● 라벨이름, 디렉토리 위치 등 초기 변수 지정

In [8]:
train_img_src_list = []

# 학습 이미지 라벨링 
class_names = ['traffic_green','traffic_red']
data_directory = '../data'

● data 폴더안에 학습하고자 하는 이미지를 학습 가능한 형태로 변형 및 학습용 데이터와 테스트용 데이터를 분리하는 전처리 부분.

In [14]:
# train data list, label list 생성
X_ = []
y_ = []

# 데이터별 path 설정
for label_idx, target in enumerate(class_names):

    target_directory = data_directory + '/' + target
    
    lst = os.listdir(data_directory)
    if '.DS_Store' in lst:
        lst.remove('.DS_Store')

    if '.ipynb_checkpoints' in lst:
        lst.remove('.ipynb_checkpoints')
    
    label_cnt = len(lst)
    
    test_cnt = 0

    for each_file in os.listdir(target_directory):
        test_cnt += 1

        # 각 train image 파일 path 설정
        file_path = '{}/{}/{}'.format(data_directory, target, each_file)

        # train image file load
        tmp_img = image.load_img(file_path, target_size=(24, 32))
        tmp_img = image.img_to_array(tmp_img)

        # Width x Height x RGB 에 해당하는 3차원 numpy 배열을 flattening
        X_.append(tmp_img)

        # one-hot encoding(labeling)
        temp_label = np.repeat(0, label_cnt)
        temp_label[label_idx] = 1
        y_.append(temp_label)

X_ = np.array(X_).astype(np.float16)
y_ = np.array(y_).astype(np.float16)

# train, valid ,test data 로 나누기 
X_train, X_temp, y_train, y_temp = train_test_split(X_, y_, test_size=0.3, random_state=100)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.3, random_state=100)

print(X_train.shape) # (현재 이미지 수, 이미지 높이크기, 이미지 폭, RGB(3))
print(X_valid.shape) # (현재 이미지 수, 이미지 높이크기, 이미지 폭, RGB(3))
print(X_test.shape)  
print(y_train.shape) # (현재 이미지 수, 라벨 수)
print(y_valid.shape) # (현재 이미지 수, 라벨 수)
print(y_test.shape)

(133, 24, 32, 3)
(39, 24, 32, 3)
(18, 24, 32, 3)
(133, 2)
(39, 2)
(18, 2)


●Custom Neural Net 모델 형성 

In [16]:
model = Sequential()
model.add(
    Conv2D(input_shape = (24, 32, 3), filters = 32, kernel_size = (3,3), strides = (3,3),
           padding = 'same', activation = 'relu')
)
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))

model.add(
    Conv2D(filters = 128, kernel_size = (3,3), strides = (1,1),
           padding = 'same', activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))

model.add(
    Conv2D(filters = 128, kernel_size = (3,3), strides = (1,1),
           padding = 'same', activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dropout(0.25))
model.add(Dense(2, activation = 'softmax'))

print('모델 구축 완료')



모델 구축 완료


● 모델 학습 실행 파트

In [ ]:
# 옵티마이저 함수 준비
opt = keras.optimizers.Adam(learning_rate = 0.001)
# 모델 컴파일링 수행
model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ['accuracy',])

# 학습 진행
hist = model.fit(X_train, y_train, validation_data = (X_valid, y_valid), epochs = 30, batch_size = 32)

# 학습된 모델 저장 경로 설정 및 저장 함수
model_path = '../model/' + 'model1.h5'
model.save(model_path)

print("모델 생성 및 저장 완료")